## Data Collection
Predicting total colony forming units (CFU) from particle counts, time of day, weather, percentage of outdoor particulates, or location.

Goal: Organize your data to streamline the next steps of your capstone.

■ Data loading
■ Data joining

In [1]:
import pandas as pd
import datetime as dt
import os
import glob
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

df1 = pd.read_excel(r'data\filt_cfus.xlsx', sheet_name = [1], header=3, dtype={'Date': dt.date}, index_col=None, parse_dates=True)
df2 = pd.read_excel(r'data\filt_cfus.xlsx', sheet_name = [2], header=1, dtype={'Date': dt.date}, index_col=None, parse_dates=True)
df3 = pd.read_excel(r'data\filt_cfus.xlsx', sheet_name = [0], index_col=None, parse_dates=True)

dfW = df1[1]
dfE = df2[2]
dfAH = df3[0]
# dfW is the first hospital's CFU air sampling data. 
# dfE is the second hospital's CFU air sampling data.
# dfAH is both hopsital's ad hoc CFU air sampling data.
df4 = pd.read_excel('data\VADIC.xlsx', sheet_name = [1], index_col=None, parse_dates=True)
df5 = pd.read_excel('data\VADIC.xlsx', sheet_name = [2], index_col=None, parse_dates=True)
df6 = pd.read_excel('data\VADMC.xlsx', index_col=None, parse_dates=True)
dfMC = pd.read_excel('data\VADMC.xlsx', index_col=None, parse_dates=True)

dfIC = df4[1]
dfAHA = df5[2]
# dfMC is both hospital's particle air sampling data.
# dfIC is Infection Control's particle air sampling data.
# dfAHA is both hopsital's ad hoc particle air sampling data.

In [2]:
# Checking for mismatched names in ad hoc data
unique_locations = dfAH['Location'].unique()
print(process.extract('ED Parking Lot', unique_locations, limit = len(unique_locations)))
print(process.extract('unused agar strip', unique_locations, limit = len(unique_locations)))

[('ED Parking lot', 100), ('ed parking lot', 100), ('ED Parking Lot', 100), ('Ed Parking Lot', 100), ('ED parking Lot', 100), ('ED parking lot', 100), ('ED Parking LOT', 100), ('ED Parking Lot ', 100), ('ED PARKING LOT', 100), ('ED PARKING LOT ', 100), ('ED Parkin Lot', 96), ('ED Parking', 95), ('ED Parking Log', 93), ('E.D. Parking Lot', 93), ('ER PARKING LOT', 93), ('E.D. PARKING LOT ', 93), ('Control ED Parking Lot', 90), ('ED Parking Lot low traffic near plants', 90), ('ED parking lot, ACC 5th floor, Baseline', 90), ('ED Parking Lot Low Traffic Near Plants', 90), ('ED Parking Lot Low traffic Near Plants', 90), ('ED Parking Lot Low Traffic', 90), ('ED Parking LOT  Low Traffic Near Plants', 90), ('ED Parking Lot by Planters', 90), ('ED parking Lot (BKGND)', 90), ('ED parking lot (control)', 90), ('ED parking lot (control for 7/12/06, for asp flavus outbreak on 7L in June)', 90), ('ED parking lot (control for OR #10 sampling)', 90), ('ED Parking Lot CONTROL', 90), ('ED', 90), ('ED L12

[('unused agar strip', 100), ('Unused Agar Strip', 100), ('Unused AGAR strip', 100), ('unused Agar strip', 100), ('Unused AGAR Strip', 100), ('Unused agar strip', 100), ('UNUSED AGAR STRIP', 100), ('UNUSED AGAR STRIP ', 100), ('Unsused Agar strip', 97), ('Unused RB Agar Strip', 95), ('unused RB Agar strip', 95), ('Unused RG Agar Strip', 95), ('Unused RB agar Strip', 95), ('unused RB Agar Strip', 95), ('Unused RB Agar strip', 95), ('Unused RB agar strip', 95), ('Unused RB AGAR strip', 95), ('Unused RB AGAR Strip', 95), ('Unused RD AGAR Strip', 95), ('Unused RB Agar Strip ', 95), ('MZ - UNUSED RB AGAR Strip', 95), ('UNUSED STRIP', 95), ('unused strip', 95), ('UNUSED AGAR STRIP 34347', 95), ('unused AGAR', 90), ('UNUSED AGAR STRIP (CONTROL)', 90), ('Unsused RB Agar Strip', 89), ('Unused RB Agar Strips', 89), ('Unusued RB Agar Strip', 89), ('Iunused RB Agar Strip', 89), ('Unused RB Strip', 88), ('Unused RB STrip', 88), ('UNUSED RB STRIP', 88), ('Blank Strip', 86), ('AIR Sample Unused RB ag

In [3]:
#Remapping categories in CFU sampling data.
matches = process.extract('unused agar strip', dfAH['Location'], limit=len(dfAH['Location']))
matches1 = process.extract('ED Parking Lot', dfAH['Location'], limit=len(dfAH['Location']))

print(matches[0:5])
print(matches1[0:5])

for match in matches:
    if match[1] > 60:
        dfAH.loc[dfAH['Location'] == match[0]] = 'unused agar strip'
        
for match in matches1:
    if match[1] >= 90:
        dfAH.loc[dfAH['Location'] == match[0]] = 'ED Parking Lot'

print(dfAH['Location'].unique())

[('unused agar strip', 100, 2), ('unused agar strip', 100, 3), ('unused agar strip', 100, 6), ('unused agar strip', 100, 16), ('unused agar strip', 100, 17)]
[('ED Parking lot', 100, 1), ('ED Parking lot', 100, 5), ('ED Parking lot', 100, 7), ('ED Parking lot', 100, 15), ('ED Parking lot', 100, 19)]
['Inside Room L108' 'ED Parking Lot' 'unused agar strip' ... 1337
 '1338 Adjacent' '1338 Sink/Fridge']


In [4]:
#Renaming and unifying column names
dfMC = dfMC.rename(columns={"Sampling Date":"Date",
                            "Sample Location":"Location",
                            "Particle (total >.3)":"Total_Particles>0.3",
                            "Particle (>.5 per m3)":"Total_Particles>0.5/m3",
                            "Time of Sampling":"Sample_Time",
                            "SF Gate Weather":"Weather",
                            "CFUs":"CFU",
                            "Percentage .3 over .5":"Ratio_0.3/0.5"})
dfIC = dfIC.rename(columns={"Sampling Date":"Date",
                            "Sample Location":"Location",
                            "Particle (total >.3)":"Total_Particles>0.3",
                            "Particle (>.5 per m3)":"Total_Particles>0.5/m3",
                            "Time of Sampling":"Sample_Time",
                            "SF Gate Weather":"Weather",
                            "CFUs":"CFU",
                            "Percentage .3 over .5":"Ratio_0.3/0.5"})
dfAHA = dfAHA.rename(columns={"Sampling Date":"Date",
                            "Sample Location":"Location",
                            "Particle (total >.3)":"Total_Particles>0.3",
                            "Particle (>.5 per m3)":"Total_Particles>0.5/m3",
                            "Time of Sampling":"Sample_Time",
                            "SF Gate Weather":"Weather",
                            "CFUs":"CFU",
                            "Percentage .3 over .5":"Ratio_0.3/0.5"})
print(dfMC.keys())
print(dfIC.keys())
print(dfAHA.keys())

Index(['Date', 'Location', 'Temp', 'RH', 'Total_Particles>0.3',
       'Total_Particles>0.5/m3', 'Sample_Time', 'Traffic', 'Weather',
       'Ratio_0.3/0.5', 'CFU', 'Notes'],
      dtype='object')
Index(['Date', 'Location', 'Temp', 'RH', 'Total_Particles>0.3',
       'Total_Particles>0.5/m3', 'Sample_Time', 'Traffic', 'Weather', 'CFU',
       'Notes'],
      dtype='object')
Index(['Date', 'Location', 'Temp', 'RH', 'Total_Particles>0.3',
       'Total_Particles>0.5/m3', 'Sample_Time', 'Traffic', 'Weather', 'CFU',
       'Notes'],
      dtype='object')


In [15]:
#converting dates
df_list = [dfW,dfE,dfAH,dfMC,dfIC,dfAHA]
for df in df_list:
    df['Date']=pd.to_datetime(df['Date'], errors='coerce')

In [16]:
#merging the particulate data now that they are in the same format
df_particulate = pd.merge(dfMC,dfIC, how='outer', on=['Date','Location'])
df_particulate = pd.merge(df_particulate,dfAHA,  how='outer', on=['Date','Location'])

In [17]:
#change to datetime
df_particulate['Date'] = pd.to_datetime(df_particulate['Date'], errors='coerce')
print(df_particulate.head())

        Date                  Location  Temp_x  RH_x  Total_Particles>0.3_x  \
0 2018-11-27                   Long 26    69.2  61.0                 3554.0   
1 2018-11-27                   Long 53    70.3  57.0                 4202.0   
2 2018-11-27   11 Long Nursing Station    71.7  57.0                 2371.0   
3 2018-11-27  11 Long Service Elevator    72.2  56.0                 9325.0   
4 2018-11-27     9 Long Nurses Station    74.0  53.0                13015.0   

   Total_Particles>0.5/m3_x Sample_Time_x  Traffic_x Weather_x  Ratio_0.3/0.5  \
0                   80565.0          1010        2.0    Cloudy       0.044113   
1                   78092.0          1015        1.0       NaN       0.053808   
2                  204947.0          1023        3.0       NaN       0.011569   
3                  480919.0          1028        1.0       NaN       0.019390   
4                  279505.0          1047        2.0       NaN       0.046564   

   ...  Notes_y Temp  RH  Total_Partic

In [18]:
df_particulate.isnull().any()

Date                        False
Location                    False
Temp_x                       True
RH_x                         True
Total_Particles>0.3_x        True
Total_Particles>0.5/m3_x     True
Sample_Time_x                True
Traffic_x                    True
Weather_x                    True
Ratio_0.3/0.5                True
CFU_x                        True
Notes_x                      True
Temp_y                       True
RH_y                         True
Total_Particles>0.3_y        True
Total_Particles>0.5/m3_y     True
Sample_Time_y                True
Traffic_y                    True
Weather_y                    True
CFU_y                        True
Notes_y                      True
Temp                         True
RH                           True
Total_Particles>0.3          True
Total_Particles>0.5/m3       True
Sample_Time                  True
Traffic                      True
Weather                      True
CFU                          True
Notes         

In [19]:
df_particulate.isnull().sum()

Date                           0
Location                       0
Temp_x                       886
RH_x                         886
Total_Particles>0.3_x        319
Total_Particles>0.5/m3_x     940
Sample_Time_x                846
Traffic_x                    324
Weather_x                    625
Ratio_0.3/0.5                940
CFU_x                       2716
Notes_x                     2689
Temp_y                      2446
RH_y                        2446
Total_Particles>0.3_y       2447
Total_Particles>0.5/m3_y    2447
Sample_Time_y               2449
Traffic_y                   2455
Weather_y                   2462
CFU_y                       2724
Notes_y                     2725
Temp                        2670
RH                          2670
Total_Particles>0.3         2676
Total_Particles>0.5/m3      2687
Sample_Time                 2682
Traffic                     2671
Weather                     2694
CFU                         2674
Notes                       2724
dtype: int

In [20]:
#ask DJ how to get the titles of columns to stop adding. (dates, dates_x, dates_y)
print(df_particulate.keys())

Index(['Date', 'Location', 'Temp_x', 'RH_x', 'Total_Particles>0.3_x',
       'Total_Particles>0.5/m3_x', 'Sample_Time_x', 'Traffic_x', 'Weather_x',
       'Ratio_0.3/0.5', 'CFU_x', 'Notes_x', 'Temp_y', 'RH_y',
       'Total_Particles>0.3_y', 'Total_Particles>0.5/m3_y', 'Sample_Time_y',
       'Traffic_y', 'Weather_y', 'CFU_y', 'Notes_y', 'Temp', 'RH',
       'Total_Particles>0.3', 'Total_Particles>0.5/m3', 'Sample_Time',
       'Traffic', 'Weather', 'CFU', 'Notes'],
      dtype='object')


In [35]:
#df_melt = pd.melt(dfW, id_vars=['Date'], var_name='Location',value_name='CFU')
df_cfu= pd.DataFrame()
df_to_merge = [dfW,dfE,dfAH]

#for df in df_to_merge:
#    df=pd.melt(df, id_vars=['Date'], var_name='Location',value_name='CFU')
print(df_to_merge)

[          Date UNUSED AGAR STRIP  15L NS  15L SEC  14L NS  14L SEC  14L PASS  \
0   2018-11-27                 0     NaN      NaN     NaN      NaN       NaN   
1   2018-10-26                 0     NaN      NaN     NaN      NaN       NaN   
2   2018-09-27                 0     NaN      NaN     NaN      NaN       NaN   
3   2018-08-27                 0     NaN      NaN     NaN      NaN       NaN   
4   2018-07-31                 0     NaN      NaN     NaN      NaN       NaN   
..         ...               ...     ...      ...     ...      ...       ...   
180 2007-12-19                 0     NaN      NaN     NaN      NaN       NaN   
181 2007-11-21                 0     NaN      NaN     NaN      NaN       NaN   
182 2007-10-26                 0     NaN      NaN     NaN      NaN       NaN   
183 2007-09-21                 0     NaN      NaN     NaN      NaN       NaN   
184 2007-08-30                 0     NaN      NaN     NaN      NaN       NaN   

     14M NS  14M SEC  11L NS  ...  6M 

In [55]:
dfE_melt = pd.melt(dfE, id_vars=['Date'], var_name='Location',value_name='CFU')
dfW_melt = pd.melt(dfW, id_vars=['Date'], var_name='Location',value_name='CFU')

In [56]:
df_CFU = pd.concat([dfE_melt,dfW_melt, dfAH])
df_CFU

,Date,Location,CFU
0,2018-09-14,Balcony BMT/Hem Onc Spine,1
1,2018-06-06,Balcony BMT/Hem Onc Spine,3
2,2018-03-02,Balcony BMT/Hem Onc Spine,NaN
3,2018-02-26,Balcony BMT/Hem Onc Spine,0
4,2017-12-22,Balcony BMT/Hem Onc Spine,5
...,...,...,...
2566,2004-02-03,1336,0 CFU
2567,2004-02-03,1337,0 CFU
2568,2004-02-03,1338 Adjacent,0 CFU
2569,2004-02-03,1338 Sink/Fridge,0 CFU


In [57]:
df_CFU['CFU'] = pd.to_numeric(df_CFU['CFU'],errors='coerce')
df_CFU['CFU'] = df_CFU['CFU'].fillna(0)

In [65]:
#merging the CFU data now that they are in the same format
df_all = pd.merge(df_particulate,df_CFU, how='outer', on=['Date','Location'])
df_all

,Date,Location,Temp_x,RH_x,Total_Particles>0.3_x,Total_Particles>0.5/m3_x,Sample_Time_x,Traffic_x,Weather_x,Ratio_0.3/0.5,...,Temp,RH,Total_Particles>0.3,Total_Particles>0.5/m3,Sample_Time,Traffic,Weather,CFU_x,Notes,CFU_y
0,2018-11-27,Long 26,69.2,61.0,3554.0,80565.0,1010,2.0,Cloudy,0.044113,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-11-27,Long 53,70.3,57.0,4202.0,78092.0,1015,1.0,NaN,0.053808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-11-27,11 Long Nursing Station,71.7,57.0,2371.0,204947.0,1023,3.0,NaN,0.011569,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-11-27,11 Long Service Elevator,72.2,56.0,9325.0,480919.0,1028,1.0,NaN,0.019390,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-11-27,9 Long Nurses Station,74.0,53.0,13015.0,279505.0,1047,2.0,NaN,0.046564,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13183,2004-02-03,1336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
13184,2004-02-03,1337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
13185,2004-02-03,1338 Adjacent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
13186,2004-02-03,1338 Sink/Fridge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [66]:
#dfE.reset_index()
dftest = df_all.set_index(['Date','Location'])
print(dftest.head())

                                     Temp_x  RH_x  Total_Particles>0.3_x  \
Date       Location                                                        
2018-11-27 Long 26                     69.2  61.0                 3554.0   
           Long 53                     70.3  57.0                 4202.0   
           11 Long Nursing Station     71.7  57.0                 2371.0   
           11 Long Service Elevator    72.2  56.0                 9325.0   
           9 Long Nurses Station       74.0  53.0                13015.0   

                                     Total_Particles>0.5/m3_x Sample_Time_x  \
Date       Location                                                           
2018-11-27 Long 26                                    80565.0          1010   
           Long 53                                    78092.0          1015   
           11 Long Nursing Station                   204947.0          1023   
           11 Long Service Elevator                  480919.0          1

In [ ]:
df

Many locations are named in differing patterns, or the room number is in the prior sample(s).

## Data Organization
Create a file structure and add your work to the GitHub repository you’ve created for this project.

In [ ]:
the_list = pd.DataFrame()
the_list = the_list.append(df) for df in df_list
    
print(the_list)

In [ ]:
result = pd.concat(df_list)

## Data Definition
Goal: Gain an understanding of your data features to inform the next steps of your project.

## Data Cleaning
Goal: Clean up the data in order to prepare it for the next steps of your project